## Estimating Free Drivable Road Space

#### Environment setup
Import dependencies, define constant values

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os

WORKING_DIR = '.'
MODELS_DIR = 'model'
VIDEO_DIR = 'video'
TEMP_DIR = 'tmp'

Set TensorFlow as Keras back-end

In [3]:
import keras
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config = config)

#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

def setTensorFlowBackend():
    session = get_session()
    keras.backend.tensorflow_backend.set_session(session)
    
setTensorFlowBackend()

ModuleNotFoundError: No module named 'keras'

#### Pre-trained Model Loading
Load pre-trained model and prepare it if needed

In [4]:
from keras_retinanet.models import load_model, convert_model

def load_model(model_name,
               backbone_name,
               should_convert_to_inference_model = False,
               should_print_summary = False):
    
    model_path = os.path.join(WORKING_DIR, MODELS_DIR, model_name)
    model = load_model(model_path, backbone_name = backbone_name)

    if (should_convert_to_inference_model):
        model = convert_model(model)
        
    if (should_print_summary):
        print(model.summary())
    
    return model

def get_name_for_label(label):
    return 'Label'

# labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

ModuleNotFoundError: No module named 'keras_retinanet'

#### Hardware Setup
Detect physical GPU

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.physical_device_desc for x in local_device_protos if x.device_type == 'GPU']

gpu = get_available_gpus()[-1][17:33]
print(gpu)

#### Debug Helper Methods

In [ ]:
import matplotlib.pyplot as plt

def show_image(image, size = (15, 15), show_axis = 'off'):
    plt.figure(figsize = size)
    plt.axis(show_axis)
    plt.imshow(image)
    plt.show()

#### Object Detection


In [ ]:
import cv2
import time
import numpy as np

from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image

def apply_region_of_interest_to_image(image):
    # TODO: Implement
    return image

def prepare_frame_for_detection(frame):
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    image = preprocess_image(frame_bgr)
    image = apply_region_of_interest_to_image(image)
    image, scale = resize_image(image)
    
    return image, scale

def detect_objects_from_prepared_image(image, image_scale, model, verbose):
    start_time = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis = 0))
    boxes /= image_scale
    if (verbose):
        print("image processing time: ", time.time() - start_time)
    
    return zip(boxes, scores, labels)

def visualize_detections(detections, image, score_threshold = 0.5):
    for box, score, label in detections:
        if (score < score_threshold):
            continue
            
        box_type = box.astype(int)
        box_color = label_color(label)
        
        draw_box(draw, box_type, color = box_color)
        caption = "{} {:.3f}".format(get_name_for_label(label), score)
        draw_caption(image, box_type, caption)
        
    return image

def detect_objects_in_frame(frame, model, verbose):
    image, image_scale = prepare_frame_for_detection(frame)
    detections = detect_objects_from_prepared_image(image, image_scale, model, verbose)
    annotated_image = visualize_detections(detections, image)
    
    return annotated_image
    
def run_objects_detection_from_capture(capture, model, verbose = False):
    frames_read = 0
    
    while (True):
        retval, frame = capture.read()
        if not retval:
            return
        
        annotated_image = detect_objects_in_frame(frame, model, verbose)
        annotated_image_path = os.path.join(WORKING_DIR, TEMP_DIR, 'img%08d.jpg' % frames_read)
        cv2.imwrite(annotated_image_path, annotated_image)
        
        if (verbose):
            show_image(annotated_image)
        
        frames_read += 1
        
    capture.release()
    cv2.destroyAllWindows()

#### Object Detection on Video Capture
Load model and video, run object detection

In [ ]:
video_path = os.path.join(WORKING_DIR, VIDEO_DIR, '')
video_capture = cv2.VideoCapture(video_path)

model = load_model(
    model_name = 'resnet50_coco_best_v2.1.0.h5',
    backbone_name = 'resnet50',
    should_print_summary = True
)

run_objects_detection_from_capture(video_capture, model)